# TV Script Generation
This project will utilize RNN's to generate plausible TV scripts. Simpson's scripts from 27 seasons will be used as input. Specifically, those scenes that take place at Moe's Tavern.

In [4]:
import helper
import numpy as np

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
text = text[81:]

In [5]:
# Exploring the data

view_sentence_range = (0, 10)

print("Dataset Stats")
print("Number of unique words (Roughly): {}".format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print("Number of scenes: {}".format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene" {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print("Number of lines: {}".format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print("Average number of words in each line: {}".format(np.average(word_count_sentence)))

print()
print("The sentences {} to {}:".format(*view_sentence_range))
print("\n".join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))


Dataset Stats
Number of unique words (Roughly): 11492
Number of scenes: 262
Average number of sentences in each scene" 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implementing Preprocessing Functions

### Lookup Table
To create a word embedding, first transform the words to ids. Following function will return a tuple containing two dictionaries: One to translate words to id's, and another to translate id's to words.

In [7]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
We need to split the script into a word array using spaces as delimiters. The punctuation used in the script would make it difficult for the network to differentiate between words like "hello!" and "hello?". The following function will tokenize the punctuation using a lookup table which will later be used to replace punctuation in the script so that they are interpreted individually instead of as a part of another word. 

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokens = {
        "."  : "||Period||",
        ","  : "||Comma||",
        "\"" : "||Quotation_Mark||",
        ";"  : "||Semicolon||",
        "!"  : "||Exclamation_Mark||",
        "?"  : "||Question_Mark||",
        "("  : "||Left_Parentheses||",
        ")"  : "||Right_Parentheses||",
        "--" : "||Dash||",
        "\n" : "||Return||"
    }
    
    return tokens

## Preprocess the data and save
Utilizing the prepreocessing functions from the helper function.

In [9]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

## Load preprocessed data
The follownig code block will reload the preprocessed data so that all the above code does not need to be re-run each time.

In [1]:
import numpy as np
import helper
import tensorflow as tf

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building Neural Net Components

### Input
following function creates placeholders for the neural network. 

In [2]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return inputs, targets, learning_rate


### Build RNN Cell 
Stack tensorflow basicLSTM cells into a MultiRNNCell. 

In [3]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    lstm_layers = 3
    
    # TF 1.1 compatible -> basic lstm in multi rnn 1 line
    
    cell = tf.contrib.rnn.MultiRNNCell([
                tf.contrib.rnn.BasicLSTMCell(rnn_size) for _ in range(lstm_layers)])
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, 'initial_state')
    
    return cell, initial_state


### Word Embedding
Apply embedding to input data using tensorflow. Returns the embedded sequence.


In [4]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embed = tf.contrib.layers.embed_sequence(input_data, 
                                             vocab_size, 
                                             embed_dim)
    
    return embed

### Build RNN
Utilize the rnn cells function to create the RNN. 

In [5]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state

### Build the Neural Network
Apply the above functions to build the network

In [6]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embedded = get_embed(input_data, 
                         vocab_size, 
                         embed_dim)
    
    out, final_state = build_rnn(cell, embedded)
    
    logits = tf.contrib.layers.fully_connected(out, 
                                                vocab_size, 
                                                activation_fn=None)
    
    return logits, final_state

## Batches

Following function will create batches of input and targets using int_text. The batches will be Numpy arrays with the shape (num_of_batches, 2, batch_size, seq_length). Each batch contains two elements:

* A single batch of input with shape [batch_size, sequence_length]
* A single batch of tarets with shape [batch_Size, sequence_length]

If the last batch cannont fill the batch size with enough data, it will be dropped. 


In [7]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_batches = len(int_text)//(batch_size*seq_length)
    inputs = np.array(int_text[:n_batches*batch_size*seq_length])
    
    # Shift elements by 1 to create targets
    targets = np.array(int_text[1:n_batches*batch_size*seq_length+1])
    targets[len(targets)-1] = inputs[0]
    
    # Reshape both
    inputs = inputs.reshape((batch_size, -1))
    targets = targets.reshape((batch_size, -1))
    
    # Create Empty output shape
    out_batches = np.zeros((n_batches, 2, batch_size, seq_length), dtype=int)
    
    # Fill output
    ctr = 0
    for n in range(0, inputs.shape[1], seq_length):
        inpt_out = inputs[:,n:n+seq_length]
        trgt_out = targets[:, n:n+seq_length]
        
        out_batches[ctr][0] = inpt_out
        out_batches[ctr][1] = trgt_out
        ctr += 1
        
    return out_batches
    

## Network Training

### Hyperparameters

In [8]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 375
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = './save'

### Build Network Graph

In [9]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, 
                                   rnn_size, 
                                   input_text,
                                  vocab_size, 
                                   embed_dim)
    
    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name="probs")
    
    # Loss function
    cost = seq2seq.sequence_loss(logits, 
                                targets, 
                                tf.ones([input_data_shape[0], input_data_shape[1]]))
    
    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)
    
    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gradients if grad is not None]
    
    train_op = optimizer.apply_gradients(capped_gradients)

## Training the network
Training the neural network on the preprocessed data.

In [10]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        
        for batch_i, (x,y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate
            }
            
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
            
            # Display infos every n batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print("Epoch {:>3} Batch {:>4}/{} train_loss = {:.3f}".format(epoch_i,
                                                                             batch_i,
                                                                             len(batches),
                                                                             train_loss))
                
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print("Model Trained and Saved")

Epoch   0 Batch    0/13 train_loss = 8.822
Epoch   3 Batch   11/13 train_loss = 6.144
Epoch   7 Batch    9/13 train_loss = 6.126
Epoch  11 Batch    7/13 train_loss = 6.165
Epoch  15 Batch    5/13 train_loss = 6.030
Epoch  19 Batch    3/13 train_loss = 6.080
Epoch  23 Batch    1/13 train_loss = 6.039
Epoch  26 Batch   12/13 train_loss = 6.074
Epoch  30 Batch   10/13 train_loss = 6.038
Epoch  34 Batch    8/13 train_loss = 5.880
Epoch  38 Batch    6/13 train_loss = 5.919
Epoch  42 Batch    4/13 train_loss = 5.339
Epoch  46 Batch    2/13 train_loss = 4.830
Epoch  50 Batch    0/13 train_loss = 4.539
Epoch  53 Batch   11/13 train_loss = 4.370
Epoch  57 Batch    9/13 train_loss = 4.247
Epoch  61 Batch    7/13 train_loss = 4.100
Epoch  65 Batch    5/13 train_loss = 3.814
Epoch  69 Batch    3/13 train_loss = 3.818
Epoch  73 Batch    1/13 train_loss = 3.617
Epoch  76 Batch   12/13 train_loss = 3.481
Epoch  80 Batch   10/13 train_loss = 3.519
Epoch  84 Batch    8/13 train_loss = 3.239
Epoch  88 B

## Save Params

In [12]:
helper.save_params((seq_length, save_dir))

## Load Params
Load model parameters for script generation

In [13]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Script Generation Functions

In [14]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_t = loaded_graph.get_tensor_by_name("input:0")
    init_state_t = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_t = loaded_graph.get_tensor_by_name("final_state:0")
    probs_t = loaded_graph.get_tensor_by_name("probs:0")
    
    return input_t, init_state_t, final_state_t, probs_t

In [15]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    size = len(int_to_vocab)
    the_word = np.random.choice(size, p=probabilities)
    output = int_to_vocab[the_word]
    
    return output

## Generate TV Script
Following code block will use the model parameters previously trained to generate a new (semi) plausible Simpson's script.

In [16]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson: so not too guy? well, you little lease i ever got us" bar no one american your brilliant man!
homer_simpson:(sobs) oh thank you, you mr. this unsourced, homer. the greatest year of the thing you got their loser? get up my pin barkeep, the sturdy sittin' in the game!
homer_simpson: i've had interested out of the game at my new game is a car, moe.
crowd: yeah, you knockin' beer?
barney_gumble: how happened to you? man, you can't do so people more.
moe_szyslak: okay, i'm like some american comedy somethin'?
fox_mulder: win this people were getting years.(to vicious) but i like to find salad house out of the bar.
lenny_leonard: no way and just do i see the great year into a way to be just leaving, ladies up for my bar.
barney_gumble: oh, i don't want anything to be fun.
homer_simpson: i'm gonna get them to turn around out for all my favorite delighted down his face is
